In [124]:
import codecs
import math
import operator
import numpy as np
from sklearn.feature_selection import SelectFdr
from sklearn.feature_selection import chi2
from scipy.sparse import csr_matrix

# https://github.com/ehsanasgari/DiTaxa

class Chi2Analysis(object):
    # X^2 is statistically significant at the p-value level
    def __init__(self, X, Y, feature_names):
        '''
        :param X: data matrix (sample (plants) x features (chr.events) )
        :param Y: 1 or 0 (phenotype)
        :param feature_names: list of X columns (chr#.#.SS - NN - SN)
        '''
        self.X = X
        self.Y = Y
        self.feature_names = feature_names


    def extract_features_fdr(self, file_name, N=-1, alpha=5e-2, direction=True, allow_subseq=True, binarization=True):
        '''
        :param file_name: output Chi2
        :param N: Top-N significant features
        :param alpha: the min p-value
        :param direction: if true the score would have sign
        :param allow_subseq:
        :param binarization: if the data is not binary ==> 'binary','median'
        :return:
        '''
        '''
            Feature extraction with fdr-correction
        '''
        # https://brainder.org/2011/09/05/fdr-corrected-fdr-adjusted-p-values/
        # Filter: Select the p-values for an estimated false discovery rate
        # This uses the Benjamini-Hochberg procedure. alpha is an upper bound on the expected false discovery rate.
        selector = SelectFdr(chi2, alpha=alpha)
        
        if binarization=='median':
            try:
                X = X.toarray()
            except:
                pass
            median_vec=np.median(X,axis=0)
            X=np.zeros(X.shape)            
            X[np.where(X>np.median(X,axis=0))]=1
            X=csr_matrix(X)
        elif binarization:
            try:
                X = X.toarray()
            except:
                pass
            X[np.where(X>0)]=1
            X=csr_matrix(X)
        else:
            X=self.X
        
        selector.fit_transform(X, self.Y)
        scores = {self.feature_names[i]: (s, selector.pvalues_[i]) for i, s in enumerate(list(selector.scores_)) if
                  not math.isnan(s)}
        if N==-1:
            scores = sorted(scores.items(), key=operator.itemgetter([1][0]), reverse=True)
        else:
            scores = sorted(scores.items(), key=operator.itemgetter([1][0]), reverse=True)[0:N]

        f = codecs.open(file_name, 'w')
        c_1 = np.sum(self.Y)
        c_0 =np.sum([1 for x in self.Y if x==0])
        f.write('\t'.join(['feature', 'score', 'p-value', 'mean+-pos', 'mean+-neg']) + '\n')
        try:
            X = X.toarray()
        except:
            pass
        pos_scores = []

        extracted_features=[]
        for w, score in scores:
            feature_array = X[:, self.feature_names.index(w)]
            pos = [feature_array[idx] for idx, x in enumerate(self.Y) if x == 1]
            neg = [feature_array[idx] for idx, x in enumerate(self.Y) if x == 0]
            m_pos=np.mean(pos)
            s_pos=np.std(pos)
            m_neg=np.mean(neg)
            s_neg=np.std(neg)
            
            c11 = np.sum(pos)
            c01 = c_1 - c11
            c10 = np.sum(neg)
            c00 = c_0 - c10
            s=score[0]
            if direction and c11 > ((1.0 * c11) * c00 - (c10 * 1.0) * c01):
                s=-s
            s=np.round(s,2)

            if allow_subseq:
                pos_scores.append([str(w), s, score[1], m_pos, m_neg])
                #if m_pos> m_neg:
                f.write('\t'.join([str(w), str(s), str(score[1])] + [str(x) for x in [m_pos, s_pos, m_neg, s_neg]]) + '\n')
            else:
                flag=False
                for feature in extracted_features:
                    if w in feature:
                        flag=True
                if not flag:
                    pos_scores.append([str(w), s, score[1], m_pos, m_neg])
                    f.write('\t'.join([str(w), str(s), str(score[1])] + [str(x) for x in [m_pos, s_pos, m_neg, s_neg]]) + '\n')

        f.close()
        return pos_scores

In [125]:
import pandas as pd

In [126]:
df = pd.read_csv('mgeno.csv')

In [127]:
df

,Unnamed: 0,id,chr1_1_1.243674,chr1_1_1.256937,chr1_1_1.257119,chr1_1_1.342929,chr1_1_1.34307,chr1_1_1.411035,chr1_1_1.411325,chr1_1_1.411438,...,chr8_8_34.715975,chr8_8_35.029057,chr8_8_35.033008,chr8_8_35.153536,chr8_8_35.153587,chr8_8_35.428008,chr8_8_35.474195,chr8_8_35.474418,chr8_8_35.479132,chr8_8_35.479379
0,3,417,NN,NN,NN,NN,NN,NN,NN,NN,...,SN,SN,SN,SN,SN,SN,SN,SN,SN,SN
1,4,95,SN,SN,SN,SN,SN,SN,SN,SN,...,SN,SN,SN,SN,SN,SN,SN,SN,SN,SN
2,5,601,NN,NN,NN,NN,NN,NN,NN,NN,...,SN,SN,SN,SN,SN,SN,SN,SN,SN,SN
3,6,576,NN,NN,NN,NN,NN,NN,NN,NN,...,SN,SN,SN,SN,SN,SN,SN,SN,SN,SN
4,7,519,NN,NN,NN,NN,NN,NN,NN,NN,...,SS,SS,SS,SS,SS,SS,SS,SS,SS,SS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,684,4,SN,SN,SN,SN,SN,SN,SN,SN,...,SS,SS,SS,SS,SS,SS,SS,SS,SS,SS
682,685,471,SN,SN,SN,SN,SN,SN,SN,SN,...,NN,NN,NN,NN,NN,NN,NN,NN,NN,NN
683,686,235,SN,SN,SN,SN,SN,SN,SN,SN,...,SN,SN,SN,SN,SN,SN,SN,SN,SS,SS
684,687,481,SS,SS,SS,SS,SS,SS,SS,SS,...,SN,SN,SN,SN,SN,SN,SN,SN,SN,SN


In [128]:
df_phenotype = pd.read_csv('leafm_pheno_data.csv') 
df_phenotype

,id,leafmargin_1ser2sm
0,417,1.0
1,95,NaN
2,601,1.0
3,576,0.0
4,519,NaN
...,...,...
681,4,NaN
682,471,NaN
683,235,NaN
684,481,NaN


In [129]:
import itertools
import numpy as np 

genotype_list_plants = df[df.columns.tolist()[2::]].values.tolist()
plants_ids = df.id.tolist()
feature_names = df.columns.tolist()[2::]
map_genotype = {'NN':[1,0,0],'SS':[0,1,0],'SN':[0,0,1],'-':[0,0,0]}

In [130]:
X_draft = np.array([list(itertools.chain(*[map_genotype[e] for e in plant_genotype])) for plant_genotype in genotype_list_plants])

In [131]:
transformed_genotype_list_plants.shape

(686, 4419)

In [132]:
phenotypes_draft = df_phenotype.leafmargin_1ser2sm.tolist()

In [133]:
available_rows = [idx for idx,v in enumerate(phenotypes_draft) if not pd.isna(v)]

In [134]:
X = X_draft[available_rows,:]

In [135]:
Y = [int(v) for idx,v in enumerate(phenotypes_draft) if not pd.isna(v)]

In [136]:
len(Y)

357

In [137]:
X.shape

(357, 4419)

In [138]:
feature_names = list(itertools.chain(*[[F"{f}_NN",F"{f}_SS",F"{f}_SN"] for f in feature_names]))

In [139]:
len(feature_names)

4419

In [140]:
ch2instance = Chi2Analysis(X, Y, feature_names)

In [141]:
res = ch2instance.extract_features_fdr('leafm_markers.csv', N=-1, alpha=5e-2, direction=True, allow_subseq=True, binarization=False)

/home/media/miniconda3/envs/quranhadith/lib/python3.9/site-packages/sklearn/feature_selection/_base.py:92: UserWarning: No features were selected: either the data is too noisy or the selection test too strict.
  warn(


In [142]:
df_results = pd.read_csv('leafm_markers.csv', delimiter='\t')

In [143]:
df_results

,,feature,score,p-value,mean+-pos,mean+-neg
chr3_3_14.831109_NN,8.79,0.003036,0.209677,0.407078,0.090129,0.286366
chr3_3_14.678883_NN,8.19,0.004220,0.217742,0.412711,0.098712,0.298276
chr3_3_14.67901_NN,8.19,0.004220,0.217742,0.412711,0.098712,0.298276
chr3_3_14.714008_NN,8.19,0.004220,0.217742,0.412711,0.098712,0.298276
chr3_3_14.62926_NN,7.26,0.007050,0.209677,0.407078,0.098712,0.298276
...,...,...,...,...,...,...
chr5_5_15.096394_SN,0.00,0.999319,0.532258,0.498958,0.532189,0.498963
chr5_5_15.176103_SN,0.00,0.999319,0.532258,0.498958,0.532189,0.498963
chr1_1_9.564236_NN,0.00,0.999518,0.266129,0.441933,0.266094,0.441914
chr8_8_10.491242_NN,0.00,0.999518,0.266129,0.441933,0.266094,0.441914


In [144]:
selected_markers = [x for x,y in df_results.index.tolist()]
selected_chi2score = [y for x,y in df_results.index.tolist()]
selected_markers_pvalues = [x for x in df_results.feature.tolist()]

In [145]:
print('\n'.join([e for e in [F"{x}\t{y}\t{z}" for x,y,z in list(zip(selected_markers,selected_chi2score,selected_markers_pvalues)) if z<0.05]]))

chr3_3_14.831109_NN	8.79	0.0030362517342743
chr3_3_14.678883_NN	8.19	0.0042195166631333
chr3_3_14.67901_NN	8.19	0.0042195166631333
chr3_3_14.714008_NN	8.19	0.0042195166631333
chr3_3_14.62926_NN	7.26	0.0070496312376999
chr3_3_14.736022_NN	7.26	0.0070496312376999
chr7_7_21.732774_SS	6.93	0.0084745991478699
chr2_2_8.352478_NN	6.37	0.0116327002243976
chr2_2_8.521443_NN	5.97	0.0145469035874397
chr3_3_14.831333_NN	5.97	0.0145469035874397
chr5_5_3.058352_NN	5.96	0.0146115989991184
chr3_3_14.62909_NN	5.94	0.0148168625473754
chr7_7_21.582025_SS	5.76	0.0164189485210371
chr8_8_28.294602_NN	5.55	0.0184665330795791
chr2_2_8.471262_NN	5.53	0.0187209266776064
chr5_5_1.2418_NN	5.51	0.0189606470773353
chr7_7_4.899442_NN	5.39	0.0202140967912785
chr3_3_14.628952_NN	5.35	0.0207713644200557
chr2_2_15.108356_NN	5.3	0.0213056061653171
chr3_3_13.639105_NN	5.3	0.0213056061653171
chr3_3_13.639405_NN	5.3	0.0213056061653171
chr5_5_1.391462_NN	5.3	0.0213056061653171
chr5_5_1.164794_NN	5.22	0.0223652662425494
chr2_

In [148]:
from nltk import FreqDist

FreqDist([x.split('_')[0] for x,y,z in list(zip(selected_markers,selected_chi2score,selected_markers_pvalues)) if z<0.05]).most_common()

[('chr2', 39),
 ('chr3', 35),
 ('chr5', 18),
 ('chr8', 9),
 ('chr7', 6),
 ('chr4', 1)]

In [146]:
from nltk import FreqDist

FreqDist([x.split('.')[0] for x,y,z in list(zip(selected_markers,selected_chi2score,selected_markers_pvalues)) if z<0.05]).most_common()

[('chr2_2_13', 12),
 ('chr3_3_14', 11),
 ('chr5_5_1', 11),
 ('chr2_2_14', 9),
 ('chr2_2_15', 8),
 ('chr3_3_18', 8),
 ('chr2_2_8', 6),
 ('chr8_8_28', 6),
 ('chr3_3_13', 6),
 ('chr3_3_15', 4),
 ('chr7_7_21', 3),
 ('chr5_5_3', 3),
 ('chr2_2_7', 3),
 ('chr8_8_15', 3),
 ('chr3_3_17', 3),
 ('chr3_3_11', 3),
 ('chr7_7_4', 2),
 ('chr5_5_0', 2),
 ('chr5_5_2', 2),
 ('chr2_2_1', 1),
 ('chr4_4_7', 1),
 ('chr7_7_12', 1)]

In [147]:
FreqDist([x.split('.')[0]+'.'+x.split('.')[1][0] for x,y,z in list(zip(selected_markers,selected_chi2score,selected_markers_pvalues)) if z<0.05]).most_common()

[('chr3_3_14.6', 5),
 ('chr2_2_13.6', 5),
 ('chr3_3_18.3', 4),
 ('chr2_2_14.3', 4),
 ('chr2_2_8.3', 3),
 ('chr2_2_15.1', 3),
 ('chr3_3_13.6', 3),
 ('chr5_5_1.1', 3),
 ('chr2_2_15.0', 3),
 ('chr3_3_13.9', 3),
 ('chr3_3_18.1', 3),
 ('chr2_2_14.4', 3),
 ('chr2_2_13.2', 3),
 ('chr3_3_14.8', 2),
 ('chr3_3_14.7', 2),
 ('chr7_7_21.5', 2),
 ('chr8_8_28.2', 2),
 ('chr5_5_1.2', 2),
 ('chr5_5_1.3', 2),
 ('chr5_5_3.3', 2),
 ('chr3_3_15.1', 2),
 ('chr8_8_28.0', 2),
 ('chr5_5_0.9', 2),
 ('chr2_2_15.2', 2),
 ('chr8_8_15.7', 2),
 ('chr5_5_1.6', 2),
 ('chr2_2_7.9', 2),
 ('chr3_3_11.4', 2),
 ('chr2_2_13.3', 2),
 ('chr2_2_13.1', 2),
 ('chr3_3_17.9', 2),
 ('chr7_7_21.7', 1),
 ('chr2_2_8.5', 1),
 ('chr5_5_3.0', 1),
 ('chr2_2_8.4', 1),
 ('chr7_7_4.8', 1),
 ('chr3_3_18.2', 1),
 ('chr2_2_14.9', 1),
 ('chr3_3_15.0', 1),
 ('chr5_5_1.4', 1),
 ('chr3_3_14.0', 1),
 ('chr2_2_1.8', 1),
 ('chr2_2_7.3', 1),
 ('chr3_3_14.9', 1),
 ('chr4_4_7.9', 1),
 ('chr7_7_12.8', 1),
 ('chr8_8_15.8', 1),
 ('chr5_5_1.5', 1),
 ('chr5_5